# Feature Engineering

In this notebook, I create features for switch (i.e. if a company did switch, what denomination did they switch to), switch type (whether a company switch up, down, or neither), and the change is the numerical variables from one period to the next. 

# Load libraries and data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
cur_path = "/content/drive/MyDrive/Colab Notebooks/Capstone/"

In [ ]:
reports = pd.read_csv(cur_path+'reports2.csv')
reports.head()

,cik,scale,year,qtr,dates,cash equivalents,total assets,total liabilities,total shareholders equity,net income
0,1750,millions,2012,4,2012-11-30,69200000.0,2.189300e+09,2.189300e+09,902800000.0,17900000.0
1,2034,thousands,2015,4,2015-12-31,54349000.0,5.243940e+08,2.385370e+08,285857000.0,8270000.0
2,2034,thousands,2016,1,2016-03-31,53093000.0,5.446000e+08,2.457490e+08,298851000.0,10424000.0
3,2098,thousands,2013,3,2013-09-30,11809000.0,7.073800e+07,3.628600e+07,34452000.0,959000.0
4,2098,thousands,2015,3,2015-09-30,4621000.0,8.658500e+07,4.287800e+07,43707000.0,1208000.0


# Switch indicator
This feature indicates the size of the switch as follows: 

*   3 - ones to millions
*   2 - thousands to millions
*   1 - ones to thousands
*   0 - no switch
*   -1 - thousands to ones
*   -2 - millions to thousands

*   -3 - milions to ones




In [ ]:
switch_lst=[]
#loop through the dataset
for i in range(len(reports)):
  cik = reports['cik'].iloc[i]
  year = reports['year'].iloc[i]
  qtr = reports['qtr'].iloc[i]
  scale = reports['scale'].iloc[i]
  if i+1 == len(reports):
    switch_lst.append(0)
    break
    # check if the cik is the same for the next row and check if the scale changes
  if (reports['cik'].iloc[i+1] == cik) and (reports['scale'].iloc[i+1]==scale):
    switch_lst.append(0)
    #if the scale changes in the next row(period), check all possibilities
  elif (reports['cik'].iloc[i+1] == cik) and (reports['scale'].iloc[i+1]!=scale):
    if (scale == 'ones') and (reports['scale'].iloc[i+1]=='thousands'):
      switch_lst.append(1)
    elif (scale == 'thousands') and (reports['scale'].iloc[i+1]=='millions'):
      switch_lst.append(2)
    elif (scale == 'millions') and (reports['scale'].iloc[i+1]=='thousands'):
      switch_lst.append(-1)
    elif (scale=='thousands') and (reports['scale'].iloc[i+1]=='ones'):
      switch_lst.append(-2)
    elif (scale == 'ones') and (reports['scale'].iloc[i+1]=='millions'):
      switch_lst.append(3)
    else:
      switch_lst.append(-3)
  else:
    switch_lst.append(0)
#insert list of switch values as column
reports.insert(1, 'switch', switch_lst)
reports.head()

,cik,switch,scale,year,qtr,dates,cash equivalents,total assets,total liabilities,total shareholders equity,net income
0,1750,0,millions,2012,4,2012-11-30,69200000.0,2.189300e+09,2.189300e+09,902800000.0,17900000.0
1,2034,0,thousands,2015,4,2015-12-31,54349000.0,5.243940e+08,2.385370e+08,285857000.0,8270000.0
2,2034,0,thousands,2016,1,2016-03-31,53093000.0,5.446000e+08,2.457490e+08,298851000.0,10424000.0
3,2098,0,thousands,2013,3,2013-09-30,11809000.0,7.073800e+07,3.628600e+07,34452000.0,959000.0
4,2098,0,thousands,2015,3,2015-09-30,4621000.0,8.658500e+07,4.287800e+07,43707000.0,1208000.0


# Switch type
This feature indicates whether or not a compnay switched up, down, or neighter as follows:
*   1 - switch up
*   0 - no switch
*   -1 - switch down





In [ ]:
switch_type=[]
for i in range(len(reports)):
  #check if switch indicator is positive or negative
  #negative indicates down, positive indicates up
  if reports['switch'].iloc[i] > 0:
    switch_type.append(1)
  elif reports['switch'].iloc[i] < 0:
    switch_type.append(-1)
  #0 if neither
  else:
    switch_type.append(0)
#insert list into dataframe as column
reports.insert(1, 'switch_type', switch_type)
reports.head()

,cik,switch_type,switch,scale,year,qtr,dates,cash equivalents,total assets,total liabilities,total shareholders equity,net income
0,1750,0,0,millions,2012,4,2012-11-30,69200000.0,2.189300e+09,2.189300e+09,902800000.0,17900000.0
1,2034,0,0,thousands,2015,4,2015-12-31,54349000.0,5.243940e+08,2.385370e+08,285857000.0,8270000.0
2,2034,0,0,thousands,2016,1,2016-03-31,53093000.0,5.446000e+08,2.457490e+08,298851000.0,10424000.0
3,2098,0,0,thousands,2013,3,2013-09-30,11809000.0,7.073800e+07,3.628600e+07,34452000.0,959000.0
4,2098,0,0,thousands,2015,3,2015-09-30,4621000.0,8.658500e+07,4.287800e+07,43707000.0,1208000.0


#Change in values
These features are numerical values which denote the change in different numerical column from one year to the next. 

In [ ]:
asset_change=[]
l_change=[]
se_change=[]
ce_change=[]
ni_change=[]
for i in range(len(reports)):
  cik = reports['cik'].iloc[i]
  #no change for the first row
  if (i ==0):
    asset_change.append(0)
    l_change.append(0)
    se_change.append(0)
    ce_change.append(0)
    ni_change.append(0)
    #calculating difference between current row and previous row if the previous row belongs to the same company
  elif (reports['cik'].iloc[i-1] == cik):
    asset_change.append(reports['total assets'].iloc[i]-reports['total assets'].iloc[i-1])
    l_change.append(reports['total liabilities'].iloc[i]-reports['total liabilities'].iloc[i-1])
    se_change.append(reports['total shareholders equity'].iloc[i]-reports['total shareholders equity'].iloc[i-1])
    ce_change.append(reports['cash equivalents'].iloc[i]-reports['cash equivalents'].iloc[i-1])
    ni_change.append(reports['net income'].iloc[i]-reports['net income'].iloc[i-1])
    #0 if the previous row was a different company
  else:
    asset_change.append(0)
    l_change.append(0)
    se_change.append(0)
    ce_change.append(0)
    ni_change.append(0)

#add columns
reports['ce_change']=ce_change
reports['asset_change']=asset_change
reports['liabilities_change']=l_change
reports['se_change']=se_change
reports['net_income_change']=ni_change

In [ ]:
reports.head()

,cik,switch_type,switch,scale,year,qtr,dates,cash equivalents,total assets,total liabilities,total shareholders equity,net income,ce_change,asset_change,liabilities_change,se_change,net_income_change
0,1750,0,0,millions,2012,4,2012-11-30,69200000.0,2.189300e+09,2.189300e+09,902800000.0,17900000.0,0.0,0.0,0.0,0.0,0.0
1,2034,0,0,thousands,2015,4,2015-12-31,54349000.0,5.243940e+08,2.385370e+08,285857000.0,8270000.0,0.0,0.0,0.0,0.0,0.0
2,2034,0,0,thousands,2016,1,2016-03-31,53093000.0,5.446000e+08,2.457490e+08,298851000.0,10424000.0,-1256000.0,20206000.0,7212000.0,12994000.0,2154000.0
3,2098,0,0,thousands,2013,3,2013-09-30,11809000.0,7.073800e+07,3.628600e+07,34452000.0,959000.0,0.0,0.0,0.0,0.0,0.0
4,2098,0,0,thousands,2015,3,2015-09-30,4621000.0,8.658500e+07,4.287800e+07,43707000.0,1208000.0,-7188000.0,15847000.0,6592000.0,9255000.0,249000.0


In [ ]:
reports.to_csv(cur_path+'10-Q_switches2.csv')